# Network classification

Classify road network into one of following labels:
- `town`
- `suburb`
- `village`

In [1]:
from blocksnet.enums import SettlementCategory
import os
import osmnx as ox
from tqdm import tqdm

graphs = []

for pp in SettlementCategory:
    folder_path = pp.value
    for filename in tqdm(os.listdir(folder_path), desc=pp.value):
        file_path = os.path.join(folder_path, filename)
        g_nx = ox.load_graphml(file_path)
        g_nx.graph['category'] = pp.value
        graphs.append(g_nx)

village: 100%|██████████| 49/49 [00:00<00:00, 54.34it/s]


In [2]:
from blocksnet.machine_learning.strategy.catboost import CatBoostClassificationStrategy
from blocksnet.analysis.network.classification.core import NetworkClassifier

strategy = CatBoostClassificationStrategy({
    'iterations':200,
    'learning_rate':0.1,
    'depth':4,
})
classifier = NetworkClassifier(strategy)

In [3]:
classifier.prepare_train(graphs)

2025-08-03 22:56:47.286 | INFO     | blocksnet.analysis.network.classification.core:_get_features_df:33 - Preprocessing graphs.
100%|██████████| 355/355 [00:02<00:00, 137.55it/s]
2025-08-03 22:56:49.870 | INFO     | blocksnet.analysis.network.classification.core:_get_features_df:36 - Calculating graphs features.
100%|██████████| 355/355 [01:08<00:00,  5.18it/s]
2025-08-03 22:57:58.426 | SUCCESS  | blocksnet.analysis.network.classification.core:_get_features_df:43 - Features are successfully built.


In [4]:
classifier.train()

0.7887323943661971

In [5]:
# strategy.save('artifacts')

## Default strategy

In [6]:
from blocksnet.analysis.network.classification.core import NetworkClassifier

classifier = NetworkClassifier.default()

In [7]:
classifier.run(graphs)

2025-08-03 22:57:58.689 | INFO     | blocksnet.analysis.network.classification.core:_get_features_df:33 - Preprocessing graphs.
100%|██████████| 355/355 [00:03<00:00, 112.52it/s]
2025-08-03 22:58:01.847 | INFO     | blocksnet.analysis.network.classification.core:_get_features_df:36 - Calculating graphs features.
100%|██████████| 355/355 [01:08<00:00,  5.15it/s]
2025-08-03 22:59:10.842 | SUCCESS  | blocksnet.analysis.network.classification.core:_get_features_df:43 - Features are successfully built.


avg_degree  leaf_nodes_proportion  link_density  avg_clustering  \
0      2.632479               0.230769      0.000019        0.044634   
1      2.667722               0.232595      0.000024        0.051160   
2      3.127436               0.103448      0.000031        0.034533   
3      2.868026               0.162554      0.000045        0.036892   
4      2.950000               0.166667      0.000035        0.025000   
..          ...                    ...           ...             ...   
350    2.508475               0.293785      0.000051        0.022599   
351    2.631579               0.175439      0.000061        0.093567   
352    2.862745               0.156863      0.000042        0.008170   
353    2.666667               0.140351      0.000015        0.084795   
354    2.774566               0.202312      0.000028        0.026975   

      density  diameter  assortativity  avg_near_neighbors_dist  \
0    0.007521        37      -0.031225                97.796244   
1    0.004228        53       0.027810                81.161861   
2    0.004696        46       0.165620                92.797600   
3    0.001539        80       0.119530                64.962786   
4    0.024790        23       0.229978               107.783917   
..        ...       ...            ...                      ...   
350  0.014253        26      -0.112127                67.816268   
351  0.023288        26      -0.046066                70.331983   
352  0.028344        16       0.046019               100.848609   
353  0.047619        21      -0.002749               128.967290   
354  0.016131        30       0.111970               101.533420   

     avg_edge_length  ratio_nodes_to_link_length  link_length_entropy  \
0         201.398576                    0.003772             2.379788   
1         166.624082                    0.004499             2.567573   
2         164.138572                    0.003896             2.831230   
3         139.840803                    0.004987             3.061517   
4         197.933830                    0.003425             2.096735   
..               ...                         ...                  ...   
350       129.970944                    0.006134             2.145329   
351       133.376494                    0.005698             2.058882   
352       181.909175                    0.003841             2.256689   
353       272.012219                    0.002757             1.704535   
354       199.022313                    0.003622             1.868175   

                       category      town    suburb   village  
0       SettlementCategory.TOWN  0.966451  0.008857  0.024693  
1       SettlementCategory.TOWN  0.978913  0.009246  0.011841  
2       SettlementCategory.TOWN  0.927734  0.038700  0.033567  
3       SettlementCategory.TOWN  0.958196  0.021080  0.020724  
4       SettlementCategory.TOWN  0.624834  0.051260  0.323906  
..                          ...       ...       ...       ...  
350     SettlementCategory.TOWN  0.549521  0.041179  0.409300  
351  SettlementCategory.VILLAGE  0.385718  0.204624  0.409657  
352  SettlementCategory.VILLAGE  0.360941  0.066698  0.572360  
353   SettlementCategory.SUBURB  0.207030  0.652064  0.140906  
354     SettlementCategory.TOWN  0.514445  0.107683  0.377872  

[355 rows x 15 columns]